# 公有云部署AppBuilderSDK
**注意⚠️：部署上云功能目前处于内测阶段，使用过程中有任何问题，欢迎提issue或微信群反馈～**


## 目标
一键将自己的服务部署到公有云上，目前支持百度智能云。部署后可以自动生成公网ip，无需额外配置。具体可以用来解决以下问题：
* 在百度云部署的WebService，可以引入到AppBuilder工作流中的API节点，形成应用中嵌套应用，构建能够解决复杂业务问题的工作流。
* 可将自己本地复杂的服务、组件、模型等，方便快捷地上云，并与AppBuilder形成联动。
* 构建自己的公网对话服务。


## 部署
### 前置操作:
* 在百度智能云完成实名认证，并开通bos服务：
  * https://cloud.baidu.com/doc/BOS/s/Jk4xttg03#%E5%BC%80%E9%80%9Abos%E6%9C%8D%E5%8A%A1
* 您需要在钱包中至少充值100元，用于bcc服务、bos服务费用
  * 云服务费用标准：https://cloud.baidu.com/product-price/bcc.html
  * 默认配置使用最便宜的bcc.e1.c2m2，并且使用按量付费
  * bos存储服务费用标准：https://cloud.baidu.com/product-price/bos.html
* 通过如下步骤获得您的AK/SK信息：
  * https://cloud.baidu.com/doc/Reference/s/9jwvz2egb
* 部署的AppBuilder >= 0.9.0

### 部署流程：
通过下面三步执行部署：
* Step1 以下面的yaml文件为模板创建config.yaml，完善并创建配置文件
  * 最小修改范围：ak、sk、admin_pass、run_cmd、local_dir、APPBUILDER_TOKEN
```yaml
bce_config:
    host: "http://bcc.bj.baidubce.com"
    bos_host: "bj.bcebos.com"
    # 前置操作获取的ak、sk
    ak: ""
    sk: ""
    spec: "bcc.e1.c2m2"
    root_disk_size_in_gb: 20
    # 服务器密码，根据实际使用设置
    admin_pass: ""
    security_group_id: ""
    zone_name: cn-bj-d

appbuilder_config:
    # 本地用来打包的代码路径，根据实际使用修改
    local_dir: "./sample"
    workspace: "/home/work/appbuilder"

    # 服务的运行命令，根据实际使用修改
    run_cmd: "python3 sample.py"

env:
    APPBUILDER_LOGLEVEL: debug
    APPBUILDER_TOKEN: ""
```
* Step2 编写自己的服务运行代码（以组件服务为例）

In [ ]:
# 创建配置中的local_dir目录
mkdir sample
touch sample/sample.py

In [ ]:
# 编写服务运行代码
# sample/sample.py
import appbuilder

component = appbuilder.Playground(prompt_template="{query}", model="ERNIE-Bot")
agent = appbuilder.AgentRuntime(component=component)
agent.serve(port=8091)

* Step3 执行部署

In [ ]:
# ./config.yaml为第一步创建的config.yaml文件
appbuilder_bce_deploy --conf ./config.yaml

# 执行后取日志"deployment finished! public ip:"后的ip为部署实例的公网ip 
# 若部署结果不符合预期，使用自己配置的"admin_pass"登陆服务器进行调试，具体参考文档：https://cloud.baidu.com/doc/BCC/s/Hkbblll70

### 示例：Component服务，接入AppBuilder工作流
示例代码

In [ ]:
import appbuilder

component = appbuilder.Playground(prompt_template="{query}", model="ERNIE-Bot")
agent = appbuilder.AgentRuntime(component=component)
agent.serve(port=8091)

示例请求

In [ ]:
# public_ip为部署获取的实例公网IP
curl --location 'http://{public_ip}:8091/chat' \
--header 'Content-Type: application/json' \
--data '{
    "message": "海淀区的面积是多少",
    "stream": false
}'

通过API节点接入Appbuilder工作流，用于创建组件
* 操作参考：https://cloud.baidu.com/doc/AppBuilder/s/glv0f48qe
* API接入调试成功

<img src="../app_builder_resources/component_api_debug.png" alt="drawing" width="800"/>

### 示例：随时随地的对话服务 
**⚠️示例仅提供demo，实际业务需增加鉴权等能力**


示例代码

In [ ]:
import appbuilder


component = appbuilder.Playground(prompt_template="{query}", model="ERNIE-Bot")

agent = appbuilder.AgentRuntime(component=component)
agent.chainlit_demo(port=8091)

示例效果

<img src="../app_builder_resources/deploy_chainlit.png" alt="drawing" width="300"/>